In [26]:
!git clone https://github.com/muxspace/facial_expressions.git

fatal: destination path 'facial_expressions' already exists and is not an empty directory.


We want to recognise the facial emotions related to the images in the github repository. For this, we shall use CNN, as this is a supervised learning task.

In [27]:
#We shall create a dictionary "data" of expression names and their concerned expression
import csv
data={}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader=csv.reader(f)
  next(reader)
  for row in reader:
    key=row[2].lower() #so no expressions are in uppercase
    if key in data:
      #We are saying that if key is already in data, then append it to its respective expression
      data[key].append(row[1])
    else:
      #If key is not there, then add it wrt to the expression in the rows of expressions.
      data[key]=[row[1]]

In [28]:
emotion_list=list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [33]:
#We shall now make training and testing directories:
import os
os.mkdir('masterr_data')
os.mkdir('masterr_data/training')
os.mkdir('masterr_data/testing')


In [34]:
for emotion in emotion_list:
  os.mkdir(os.path.join('masterr_data/training', emotion))
  os.mkdir(os.path.join('masterr_data/testing', emotion))

In [37]:
#We shall now make 8 different directory for 8 different emotions
from shutil import copyfile
split_size=0.8

for emotion, images in data.items():
  train_size=int(split_size*len(images))
  train_images=images[:train_size]
  test_images=images[train_size:]
  for image in train_images:
    source=os.path.join('/content/facial_expressions/images', image)
    destination=os.path.join('/content/masterr_data/training',emotion, image)
    copyfile(source, destination)
  for image in test_images:
    source=os.path.join('/content/facial_expressions/images', image)
    destination=os.path.join('/content/masterr_data/testing',emotion, image)
    copyfile(source, destination)

In [38]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten

In [39]:
model=tf.keras.Sequential([
    Conv2D(16,(3,3), activation="relu", input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3), activation="relu"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512,activation="relu"),
    Dense(8,activation="softmax")
])
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['acc'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [40]:
train_dir='/content/masterr_data/training'
test_dir='/content/masterr_data/testing'
train_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(
    train_dir, 
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)
test_datagen=ImageDataGenerator(rescale=1.0/255)
test_generator=test_datagen.flow_from_directory(
    test_dir, 
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [41]:
es=EarlyStopping(monitor='val_acc', patience=2, min_delta=0.01)

In [42]:
model.fit_generator(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=test_generator,
                    callbacks=[es]
)

<ipython-input-42-f5bdca6f15c7>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
86/86 [==============================] - 25s 182ms/step - loss: 1.1266 - acc: 0.5817 - val_loss: 1.5106 - val_acc: 0.6120
Epoch 2/10
86/86 [==============================] - 15s 177ms/step - loss: 0.6136 - acc: 0.7930 - val_loss: 1.7247 - val_acc: 0.6495
Epoch 3/10
86/86 [==============================] - 16s 185ms/step - loss: 0.5073 - acc: 0.8221 - val_loss: 1.2186 - val_acc: 0.6630
Epoch 4/10
86/86 [==============================] - 15s 177ms/step - loss: 0.4443 - acc: 0.8393 - val_loss: 1.8273 - val_acc: 0.6652
Epoch 5/10
86/86 [==============================] - 15s 175ms/step - loss: 0.3841 - acc: 0.8626 - val_loss: 1.8913 - val_acc: 0.6678
